# Flight Delays (Brazil)

The microdata from ANAC [(National Agency for Civil Aviation)](http://www.anac.gov.br/assuntos/dados-e-estatisticas/historico-de-voos) contains all flights that departure from brazilian airports.

ANAC has available to download flight data from jan,2000 to aug,2017: [http://www.anac.gov.br/assuntos/dados-e-estatisticas/historico-de-voos](http://www.anac.gov.br/assuntos/dados-e-estatisticas/historico-de-voos).

In Brazil, it is estimated that 20% of the flights have some kind of delay[1].

[1] 
---